In [1]:
import os
import math
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from IPython.display import display
from PIL import Image

In [2]:
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.
    The angle should be given in radians.
    
    modified from answer here: https://stackoverflow.com/questions/34372480/rotate-point-about-another-point-in-degrees-python
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return int(qx), int(qy)


def coords2img(im, x, y, w, h, r):
    center = (x, y)
    width = w
    height = h
    degrees = r

    # topleft
    pt1 = [center[0] - width/2, center[1] - height/2]
    # btmleft
    pt2 = [center[0] - width/2, center[1] + height/2]
    # btmright
    pt3 = [center[0] + width/2, center[1] + height/2]
    # topright
    pt4 = [center[0] + width/2, center[1] - height/2]
    rectangle = [pt1, pt2, pt3, pt4, pt1]

    rectangle_rotated = [rotate(center, pt,  math.radians(degrees)) for pt in rectangle]

    rectangle = np.array(rectangle)
    rectangle_rotated = np.array(rectangle_rotated)

    # these are what you need
    x_min, y_min = np.min(rectangle_rotated,axis=0)
    x_max, y_max = np.max(rectangle_rotated,axis=0)

    
    return im[y_min:y_max, x_min:x_max]

In [3]:
def split(data):
    train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle = False)
    train, val = train_test_split(train, test_size=0.2, random_state=42, shuffle = False)
    
    return [train, test, val]

In [4]:
def resize(im):
    
    if im.size >40:
        val = [0.6, 0.7, 0.8]
        rand_val = val[random.randint(0,2)]
        resized = cv2.resize(im, (int(im.shape[0]*rand_val), int(im.shape[1]*rand_val)) , interpolation = cv2.INTER_AREA)
        resized2 = cv2.resize(resized, (int(resized.shape[0]*1.2), int(resized.shape[1]*1.2)) , interpolation = cv2.INTER_AREA)
        return resized2
    
    else:
        return im

In [5]:
import random

def data_aug(im):
    
    img = im.copy()
    #print(img.shape[0]/img.shape[1])
    if img.shape[0]/img.shape[1] <=0.7:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    
    
    random_size = [0.06, 0.07, 0.08, 0.09, 0.1]
    w = int(img.shape[1]*random_size[random.randint(0,4)])
    #w = int(img.shape[1]*0.06)
    h = int(w*0.6)
   
    black = [40, 50, 60, 80, 100]
    black[random.randint(0,4)]
    
    id_pic = 255 * np.ones((h,w,3), np.uint8)

    id_pic[0:int(h*0.15),:] = black[random.randint(0,4)]
    id_pic[int(h*0.95):,:] = black[random.randint(0,4)]
    id_pic[:, int(w*0.95):] = black[random.randint(0,4)]
    id_pic[:, 0:int(w*0.05)] = black[random.randint(0,4)]

    id_pic[0:int(h*0.65), int(w*0.2):int(w*0.34)] = black[random.randint(0,4)]
    id_pic[0:int(h*0.65), int(w*0.4):int(w*0.56)] = black[random.randint(0,4)]
    id_pic[0:int(h*0.65), int(w*0.64):int(w*0.80)] = black[random.randint(0,4)]
    
    id_pic = resize(id_pic)
    
    height = img.shape[0]
    width = img.shape[1]
    #cropped = test[int(height/4):int(height/4)*2, int(width/4):int(width/4)*2]
    
    y_min = int(height/4)
    y_max = int(height/4)*2
    x_min = int(width/4)
    x_max = int(width/4)*2
    
    x_rand = random.randint(x_min, int(x_max*0.8))
    y_rand = random.randint(y_min, int(y_max*0.8))
    
    rotation = [0, 0, 5, 10, 15, 20]
    
    if random.randint(0, 1) == 0:
        mag = 1
    else:
        mag = -1
    rotate_val = mag*rotation[random.randint(0, 5)]
    id_pic = Image.fromarray(id_pic).rotate(rotate_val, expand=1, fillcolor = (255,255,255))
    
    id_pic = np.array(id_pic)

    img[y_rand:y_rand+id_pic.shape[0], x_rand:x_rand+id_pic.shape[1]] = id_pic
    
    
    return img

In [6]:
def compress(image, path):
    
    val = random.randint(0,3)
    if val == 0:
        reduce = random.randint(5, 10)
    elif val == 1:
        reduce = random.randint(10, 20) 
    elif val == 2:
        reduce = random.randint(20, 30) 
    else:
        reduce = random.randint(50, 60)
        
    image = Image.fromarray(image)
    image.save(path, 
                "JPEG", 
                optimize = True, 
                quality = reduce)
    return

In [7]:
def store_im(data, file_name):
    
    counter = 0
    im_type = ['train', 'test', 'val']
    for count, path in enumerate(data):
        for file_path in path:
            #images
            im = cv2.imread(file_path)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            #print(file_path)
            txt_file = file_path[:-3]+'txt'
            txt = os.path.join(file_path, txt_file)
            print(txt_file)
            box  = []
            with open(txt, "r") as f:
                lines = f.read().splitlines()
                for detection in lines:
                    box.append([float(i) for i in detection.split()[1:]])
            for count2, points in enumerate(box):
                #print(points, count)
                pos = coords2img(im, points[0], points[1], points[2], points[3], points[4])
                neg = data_aug(pos)

                
                im_name = file_name+'_'+str(count2)+'_'+str(counter)+'.jpg'
                
                compress(pos, "train_test_val10/"+str(im_type[count])+"/0/"+file_name+'_'+str(count2)+'_'+str(counter)+'.jpg')
                compress(neg, "train_test_val10/"+str(im_type[count])+"/1/"+file_name+'_'+str(count2)+'_'+str(counter)+'.jpg')

                counter+=1
                
            

In [8]:
HABBOF_path = 'D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/'

paths = ["Lab1", "Lab2", "Meeting1", "Meeting2"]

for count, p in enumerate(paths):
    data_arr = []
    for fname in os.listdir(HABBOF_path + p):
        full_file = HABBOF_path + p + "/" + fname
        if fname[-3:] == 'jpg':
            data_arr.append(full_file)
    
    
    store_im(split(data_arr), p)

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000001.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000002.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000003.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000004.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000005.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000006.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000007.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000008.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000009.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000010.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000011.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000012.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000013.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000014.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HAB

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000122.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000123.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000124.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000125.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000126.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000127.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000128.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000129.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000130.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000131.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000132.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000133.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000134.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000135.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HAB

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000245.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000246.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000247.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000248.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000249.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000250.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000251.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000252.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000253.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000254.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000255.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000256.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000257.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000258.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HAB

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000369.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000370.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000371.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000372.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000373.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000374.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000375.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000376.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000377.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000378.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000379.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000380.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000381.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000382.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HAB

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000491.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000492.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000493.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000494.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000495.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000496.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000497.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000498.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000499.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000500.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000501.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000502.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000503.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Lab1/000504.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HAB

...

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000575.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000576.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000577.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000578.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000579.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000580.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000581.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000582.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000583.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000584.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000585.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000586.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000587.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000692.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000693.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000694.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000695.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000696.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000697.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000698.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000699.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000700.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000701.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000702.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000703.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000704.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000987.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000988.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000989.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000990.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000991.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000992.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000993.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000994.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000995.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000996.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000997.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000998.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000999.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001104.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001105.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001106.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001107.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001108.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001109.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001110.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001111.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001112.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001113.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001114.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001115.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001116.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/001

D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000815.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000816.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000817.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000818.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000819.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000820.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000821.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000822.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000823.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000824.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000825.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000826.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000827.txt
D:/Machine Learning/yolo/Footfallcam/HABBOF2/HABBOF/Meeting2/000

In [4]:
import cv2
import numpy as np
from PIL import Image
test = cv2.imread(r'D:\Machine Learning\yolo\Footfallcam\pic5.png')
test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)

import random
black = [40, 50, 60, 80, 100]
black[random.randint(0,4)]

w = int(test.shape[1]*0.2)
h = int(w*0.6)

whiteFrame = 255 * np.ones((h,w,3), np.uint8)

whiteFrame[0:int(h*0.15),:] = black[random.randint(0,3)]
whiteFrame[int(h*0.95):,:] = black[random.randint(0,3)]
whiteFrame[:, int(w*0.95):] = black[random.randint(0,3)]
whiteFrame[:, 0:int(w*0.05)] = black[random.randint(0,3)]

whiteFrame[0:int(h*0.65), int(w*0.14):int(w*0.28)] = black[random.randint(0,3)]
whiteFrame[0:int(h*0.65), int(w*0.40):int(w*0.56)] = black[random.randint(0,3)]
whiteFrame[0:int(h*0.65), int(w*0.67):int(w*0.80)] = black[random.randint(0,3)]

In [48]:
x = Image.fromarray(whiteFrame)

In [50]:
whiteFrame.shape[1::-1]

(25, 15)

In [51]:
Image.fromarray(rotate_image(whiteFrame,-10))

In [80]:
x1 = x.rotate(-20, expand=1, fillcolor = (255,255,255))
x2 = np.array(x1)

In [81]:
x2.shape

(23, 29, 3)

In [82]:
x1